In [1]:
COLLECTION_NAME = 'movies_vecdb'
DIMENSION = 384
URI = 'http://localhost:19530'

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.milvus import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
import os

docs = []

for file in os.listdir('./movie_text_data/'):
    loader = TextLoader(f'./movie_text_data/{file}')
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)  # DA OTTIMIZZARE
all_splits = text_splitter.split_documents(docs)

In [4]:
import torch

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': device}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [5]:
vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={'host':'127.0.0.1', 'port':'19530'},
    drop_old=False
)

In [ ]:
vector_db.from_documents(
    all_splits,
    embeddings,
    connection_args={'host':'127.0.0.1', 'port':'19530'},
)

In [10]:
query = "Which are the Crime movies directed by Martin Scorsese?"
result = vector_db.similarity_search(query, k=3)

In [11]:
for n, doc in enumerate(result):
    print(f"DOCUMENT {n+1}:")
    print(doc.page_content)
    print()

DOCUMENT 1:
IMDB ID -> 0031014
TITLE -> 6,000 Enemies
DIRECTED BY -> George B. Seitz
GENRES -> Crime, Drama, Romance
PLOT -> A tough prosecutor who has sent dozens of criminals to prison finds himself framed on a bribery charge and winds up in prison himself.
YEAR -> 1939
STARS -> Walter Pidgeon, Rita Johnson, Paul Kelly

DOCUMENT 2:
IMDB ID -> 1931533
TITLE -> Seven Psychopaths
DIRECTED BY -> Martin McDonagh
GENRES -> Comedy, Crime
PLOT -> A struggling screenwriter inadvertently becomes entangled in the Los Angeles criminal underworld after his oddball friends kidnap a gangster's beloved Shih Tzu.
YEAR -> 2012
STARS -> Colin Farrell, Woody Harrelson, Sam Rockwell

DOCUMENT 3:
IMDB ID -> 0037622
TITLE -> Crime, Inc.
DIRECTED BY -> Lew Landers
GENRES -> Crime, Drama, Film-Noir
PLOT -> A crusading reporter uses his friendship with a mobster to expose the machinations of organized crime in his city.
YEAR -> 1945
STARS -> Leo Carrillo, Tom Neal, Martha Tilton



In [6]:
print(f"Default collection name - {vector_db.collection_name}")
print(f"Default search params - {vector_db.search_params}")
print(f"Default index params - {vector_db.index_params}")

Default collection name - LangChainCollection
Default search params - {'metric_type': 'L2', 'params': {'ef': 10}}
Default index params - None


In [7]:
vector_db.col

<Collection>:
-------------
<name>: LangChainCollection
<description>: 
<schema>: {'auto_id': True, 'description': '', 'fields': [{'name': 'source', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}

In [12]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [13]:
llm = LlamaCpp(
        model_path="./models/llama-2-13b-chat.ggmlv3.q4_1.bin",
        n_gpu_layers=-1,
        n_batch=512,
        temperature=0.1,
        n_threads=8,
        max_tokens=16384,
        # f16_kv=True,
        # callback_manager=CallbackManager(StreamingStdOutCallbackHandler()),
        verbose=True
)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060, compute capability 8.6
llama.cpp: loading model from ./models/llama-2-13b-chat.ggmlv3.q4_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 3 (mostly Q4_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: using CUDA f

In [17]:
from langchain_core.prompts import PromptTemplate

template = """You are an helpful AI assistant, which is expert about the field of Movies and Cinema.
Answer to the question in a precise way: use the search results to build the answer, if the asked informations
don't exist within the search results, DON'T INVENT THEM AND JUST ANSWER 'I don't know how to answer to this question'. 

For each movie, report these informations following this structure:
- Title
- Year
- Genre
- Director
- Brief description of the plot
- Main actors

Generate a maximum of 5 movies.
At the end of your whole answer, you must generate the special token '[END]'. Do not generate other tokens after this one.

Question: {question}

Answer: """

prompt = PromptTemplate.from_template(template)

In [18]:
from langchain_core.runnables import RunnablePassthrough

retriever = vector_db.as_retriever()

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
)

In [22]:
resp = rag_chain.invoke("Report me one Crime movie or with Johnny Depp")
print(str(resp))

Llama.generate: prefix-match hit




Title: Donnie Brasco (1997)
Year: 1997
Genre: Crime, Drama
Director: Mike Newell
Brief description of the plot: The film is based on the true story of Joseph D. Pistone, an FBI agent who infiltrated the mafia in the 1970s and 1980s. Johnny Depp plays the role of Benjamin "Lefty" Ruggiero, a mobster who becomes a close friend of Pistone's (played by Al Pacino).
Main actors: Al Pacino, Johnny Depp, Michael Madsen, Bruno Kirby, and James Russo.

[END]



llama_print_timings:        load time = 10645.35 ms
llama_print_timings:      sample time =    55.32 ms /   160 runs   (    0.35 ms per token,  2892.47 tokens per second)
llama_print_timings: prompt eval time =   642.33 ms /    12 tokens (   53.53 ms per token,    18.68 tokens per second)
llama_print_timings:        eval time =  9193.50 ms /   159 runs   (   57.82 ms per token,    17.29 tokens per second)
llama_print_timings:       total time = 10204.97 ms
